<div class="alert alert-block alert-info" style="margin-top: 20px">
    <a href="https://cocl.us/exploring_spark_graphX">
         <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/BD0223EN/Ad/TopAd.png" width="750" align="center">
    </a>
</div>

# <center>Exploring GraphX</center>
## <center>Visualizing GraphX and Exploring Graph Operators</center>

<img src="http://spark.apache.org/docs/latest/img/graphx_logo.png" width=300 />

## Welcome to the second lab in the course, Exploring GraphX.

### GraphX is Apache Spark's API for graph and graph-parallel computations.

In the last exercise, you looked at an introduction to GraphX, specifically how to create the components that make up a Graph. Then fully create a Graph given the information of the vertices and edges. In this lab, you will get more practice constructing a GraphX, extracting information using Graph Operators, and look at visualization of the Graph.

So in the last exercise, you looked at creating our simple recreation of "facebook". You were given most of the code, so let's go ahead and recreate the same graph with a little less help and a bit more intitution!

First we will import the following libraries:

- org.apache.spark._ 
- org.apache.spark.graphx._
- org.apache.spark.rdd.RDD 

In [1]:
// Type your code here
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

In our "facebook" graph we created we had the following People:

- Billy Bill -> VertexId = 1
- Jacob Johnson -> VertexId = 2
- Andrew Smith -> VertexId = 3

and 2 Pages:

- Iron Man Fan Page -> VertexId = 4
- Captain America Fan Page -> VertexId = 5

And we are going to create the vertices in one step! This will be tied to the variable called vertexRDD



In [4]:
// Type your code here
val vertexRDD: RDD[(Long, (String, String))] = sc.parallelize(Array((1L, ("Billy Bill", "Person")), 
                                                                    (2L, ("Jacob Johnson", "Person")), 
                                                                    (3L, ("Andrew Smith", "Person")),
                                                                    (4L, ("Iron Man Fan Page", "Page")), 
                                                                    (5L, ("Captain America Fan Page", "Page"))))

vertexRDD = ParallelCollectionRDD[1] at parallelize at <console>:35


ParallelCollectionRDD[1] at parallelize at <console>:35

Awesome! Now let's create the same relationships in one step too:

- Billy is Friends with Jacob
- Billy is Friends with Andrew
- Jacob is a Follower of the Iron Man Fan Page
- Jacob is a Follower of the the Captain America Fan Page
- Andrew is a Follower of the the Captain America Fan Page

This edge will be called edgeRDD.

Hint: The Type is RDD[Edge[String]]

In [5]:
// Type your code here

val edgeRDD: RDD[Edge[String]] = sc.parallelize(Array(Edge(1L, 2L, "Friends"), Edge(1L, 3L, "Friends"), 
                                                      Edge(2L, 4L, "Follower"), Edge(2L, 5L, "Follower"), 
                                                      Edge(3L, 5L, "Follower")))

edgeRDD = ParallelCollectionRDD[2] at parallelize at <console>:36


ParallelCollectionRDD[2] at parallelize at <console>:36

Now let's create the a variable called <code>defaultvertex</code> which will be the "fallback" for any edges that cannot connect to a vertex. It is only a tuple which contains "Self" and "Missing"

In [6]:
// Type your code here
var defaultvertex = ("Self", "Missing")

defaultvertex = (Self,Missing)


(Self,Missing)

Alright, now let's go ahead and construct the Graph! We will name it facebook again!

In [7]:
// Type your code here
var facebook = Graph(vertexRDD, edgeRDD, defaultvertex)

facebook = org.apache.spark.graphx.impl.GraphImpl@60ca47a


org.apache.spark.graphx.impl.GraphImpl@60ca47a

Perfect! Here's a reminder of the visualized Graph:

<img src = "http://i.imgur.com/rhkiopM.png">

Alright so now we will take a look at the few of the Graph Operators! These Graph Operators are called by using the Graph "facebook" variable we created. You use them by calling them on the Graph variable or "facebook" in our case. Let's try to extract how many vertices there are in this graph by using <code>numVertices</code> function.

In [8]:
// Type your code here
facebook.numVertices

5

Sweet! Now let's find out the number of edges using <code>numEdges</code> function.

In [9]:
// Type your code here
facebook.numEdges

5

Ironically, they are both the same. So make sure you didn't just use the same function both times! Haha.

Now the next Operator we will look at involve degrees. In this case we are talking about degrees as the number of edges a vertex touches! The Edges in a multi-directional graph have a direction. As you can see, sometimes it can be mutual such as:

-> Billy is a Friend of Andrew. 

-> Andrew is a Friend of Billy.

However there are cases where the edge or "relationship" is not mutual. This is such as:

-> Jacob is a Follower of the Captain America Fan Page.

-> Captain America Fan Page is a Follower of Jacob.

So, if we are looking at a specific vertex, we can determine the edges that point "out" with the function <code>outDegrees</code>. However, the question is... How do we find a specific vertex? We use the filter function like we did in the last exercise!

We can use the filter function on the <code>outDegrees</code> function of facebook and select the case where the id is the number or numbers we want.

Let's find Billy's outDegrees information by filtering it with a id of 1 and using the collect function afterwards. Let's save it as <code>Billy_outDegree</code>.

Note: The case we will need is case(id, outdegree), as the id of the person is the first parameter and the outdegree number is the second parameter.

In [10]:
// Type your code here
var Billy_outDegree = facebook.outDegrees.filter{ case(id, outdegree) => id == 1}.collect

Billy_outDegree = Array((1,2))


Array((1,2))

Awesome! Now let's go ahead print out Billy_outDegree. However, we will need to index it using () brackets instead of [] brackets. The index should be 0.

In [11]:
// Type your code here
print(Billy_outDegree(0))

(1,2)

So the result should contain the id (first parameter) and the outDegree of that parameter in the second index. Therefore the outDegree of Billy is 2.

Now let's do the same for Billy but lets find it's inDegree. We use <code>Billy_inDegree</code> as the variable. Then print the first index of <code>Billy_inDegree</code> like before

In [12]:
// Type your code here
var Billy_inDegree = facebook.inDegrees.filter{ case(id, outdegree) => id == 1}.collect
print(Billy_inDegree(0))

Name: java.lang.ArrayIndexOutOfBoundsException
Message: 0
StackTrace: 

You got an error when you tried to print the <code>Billy_inDegree</code> didn't you? That's to be expected because since there wasn't an inDegree value for Billy's vertex, there wasn't an anything in <code>Billy_inDegree</code> variable.

Now let's take a look at the degrees operator. We will do something different than before, and go ahead and use a for loop to cycle through the total degree of each vertex (inDegree + OutDegree)

In [13]:
// Type your code here
for (degree <- facebook.degrees.collect) 
{
    println(degree)
}

(1,2)
(2,3)
(3,2)
(4,1)
(5,2)


lastException: Throwable = null


So the list that shows up is the same format as before, the first element is the vertex id and the second element is the number of degrees corresponding to that element.

Now the next Graph Operators we are looking at is .vertices, .edges, and .triplets. As you have used, and seem them before in the last exercise. They are Graph Operators and it is important to know how to use each of their cases:

- .vertices -> Uses format of the defined Vertices of the graph. <br>
Ex. We defined our Vertices as (Long, (String, String)), therefore when you call a case on this, you must define variables for each such as (id, (name, user_type)).

- .edges -> Uses format of the defined Edges of the graph. <br>
Ex. We defined our Edges as Edge[String], therefore when you call a case on this, you can just define one variable such as (relation). However, this variable will have attributes such as .srcId (Source Id), .dstId (Destination Id), and .attr (Attribute).

- .triplets -> Uses the combined format of the defined Vertices and Edges. <br>
Ex. Follow the above example, when we call a case on this, you define one variable such as (triplet). And this variable will have attributes of both Vertices and Edges such as .srcAttr (Source Attribute), .dstAttr (Destination Attribute) from Vertices, and .srcId (Source Id), .dstId (Destination Id), and .attr (Attribute) from Edges.

So since you've dealt with .vertices and edges, we do a quick example with each then start looking at how to visualize the graph with .triplets since it a combination of .vertices and .edges.

Unfortunately, GraphX does not have any build-in visualization, so it's important to know how to create views. Let's go ahead and trying printing out all of the vertices.

Hint: Use a for loop and the collect function on .vertices

In [14]:
// Type your code here
for (vertex <- facebook.vertices.collect) {
    println(vertex)
}

(1,(Billy Bill,Person))
(2,(Jacob Johnson,Person))
(3,(Andrew Smith,Person))
(4,(Iron Man Fan Page,Page))
(5,(Captain America Fan Page,Page))


Awesome! Now let's do the same with edges just so we have an idea of all the vertices and edges.

In [15]:
// Type your code here
for (edge <- facebook.edges.collect) {
    println(edge)
}

Edge(1,2,Friends)
Edge(1,3,Friends)
Edge(2,4,Follower)
Edge(2,5,Follower)
Edge(3,5,Follower)


Alright, now let's use triplets to create a view of the graph. Just like in last two examples, we will use the collect function on .triplets, however we will denote the Source Attribute (.srcAttr), the edge attribute (.attr), and the Destination Attribute (.dstAttr) all in the same <code>println</code> statement to denote each relationship.

Hint: Make sure to use the index on the Source and Destination Attribute!

In [16]:
// Type your code here
for (triplet <- facebook.triplets.collect) {
    print(triplet.srcAttr._1)
    print(" is a ")
    print(triplet.attr)
    print(" of ")
    println(triplet.dstAttr._1)
}

Billy Bill is a Friends of Jacob Johnson
Billy Bill is a Friends of Andrew Smith
Jacob Johnson is a Follower of Iron Man Fan Page
Jacob Johnson is a Follower of Captain America Fan Page
Andrew Smith is a Follower of Captain America Fan Page


The view looks great! It is important to know how to create a view because GraphX does not have any visualization built-in, it is mainly a parallel graph processing library. There are alternatives such as Graphlab and Gephi, but we won't be looking into these in this course.

So the only issue we have is that the relation "Friends" and "Follower" is different. One is plural, and one is singular. So when we print the view, one set will be correct while the other set will be grammatically incorrect. 

You may have noticed that the visualization is correct, but not the actual graph. That is not a mistake - the visualization is what we want it to be, but it was left there as we want to change this error by learning how to! 

We will want to change this in the Graph to make "Friends" singular to "Friend". We will be exploring how to do this in the next lab, so starting thinking!

Now we will take a look at an important algorithm in GraphX: pageRank.

Pagerank is a algorithm that measures the importance of each vertex by directly correlating it's importance with edges (properties and quantity). There are two options for Pagerank, static and dynamic. Static runs for a fixed number of iterations while dynamic runs until the rank converges.

We won't worry too much as we will just introduce the concept. Now, in this case I went ahead and used the pageRank function on our graph, and collected the vertices into a variable called rank. Now go ahead and try to print it out!

Note: rank is a collection, so you will need to use a for loop!

In [17]:
val rank = facebook.pageRank(0.1).vertices.collect

rank = Array((1,0.6622516556291391), (2,0.9437086092715232), (3,0.9437086092715232), (4,0.9437086092715232), (5,1.5066225165562912))


Array((1,0.6622516556291391), (2,0.9437086092715232), (3,0.9437086092715232), (4,0.9437086092715232), (5,1.5066225165562912))

In [18]:
// Type your code here
for (rankee <- rank) {
    println(rankee)
}

(1,0.6622516556291391)
(2,0.9437086092715232)
(3,0.9437086092715232)
(4,0.9437086092715232)
(5,1.5066225165562912)


Alright! So what do these numbers mean? The first part is the ID of the vertex, and the second value is their rank determined by pageRank. The higher the number, the higher the rank. So it looks like ID = 5 ("Iron Man Fan Page") is the most important, which makes sense because it has two followers (the most). 

So this is just an introduction to pageRank, so if you would like to dive more into it, feel free to take a look at this documentation! http://spark.apache.org/docs/latest/graphx-programming-guide.html#pagerank

That's it for this lab, in the next exercise we will be taking a look at modifying the graph and how GraphX does it with RDDs which are immutable!